In [2]:
from google.colab import files
import pandas as pd

# Faz upload do arquivo do seu PC
uploaded = files.upload()

# Pega o nome do arquivo enviado
file_path = list(uploaded.keys())[0]


Saving Dados_2024-2025 v3.xlsx to Dados_2024-2025 v3.xlsx


In [4]:
#Lista planilhas do arquivo
xls = pd.ExcelFile(file_path)
print("Planilhas do arquivo:")
for sheet_name in xls.sheet_names:
    print(sheet_name)

Planilhas do arquivo:
CTBA_2024
CTBA_2025
RM_2024
1_SEM_2024_RM
1_SEM_2025_RM
2_SEM_2024_RM
RM_2025
GERAL_2024
GERAL_2025


In [5]:
#selecionar planilha
base_2024 = ['CTBA_2024']
base_2025 = ['CTBA_2025']

In [6]:
# Ler apenas as planilha 2024
dt = pd.read_excel(file_path, sheet_name=base_2024)
base = base_2024

In [ ]:
# Ler apenas planilha 2025
dt = pd.read_excel(file_path, sheet_name=base_2025)
base = base_2025

In [7]:
# Lista de crimes perigosos
crimes_perigosos = [
    'FURTO SIMPLES',
    'FURTO QUALIFICADO',
    'ROUBO',
    'DROGAS PARA O CONSUMO PESSOAL',
    'DANO',
    'ROUBO AGRAVADO',
    'VIOLACAO DE DOMICILIO',
    'FURTO DE COISA COMUM',
    'PORTE ILEGAL DE ARMA DE FOGO, ACESSORIO OU MUNICAO - USO PERMITIDO',
    'EXTORSAO MEDIANTE SEQUESTRO',
    'ROUBO COM RESULTADO DE LESAO CORPORAL GRAVE',
    'COMERCIO ILEGAL DE ARMA DE FOGO'
]

df_crimes_perigosos = dt[base[0]][dt[base[0]]['Natureza'].isin(crimes_perigosos)]



In [8]:
# Lista de tipos de furtos

furtos = [
    'FURTO SIMPLES', 'FURTO QUALIFICADO', 'FURTO DE COISA COMUM',
]

In [9]:
# Lista de tipos de roubos

roubos = [
    'ROUBO', 'ROUBO AGRAVADO','ROUBO COM RESULTADO DE LESAO CORPORAL GRAVE'
]

In [10]:
furtos_roubos = [
    'FURTO SIMPLES', 'FURTO QUALIFICADO', 'FURTO DE COISA COMUM','ROUBO', 'ROUBO AGRAVADO','ROUBO COM RESULTADO DE LESAO CORPORAL GRAVE'
]

In [11]:
crimes_violentos = [
    'ROUBO','ROUBO AGRAVADO','EXTORSAO MEDIANTE SEQUESTRO','ROUBO COM RESULTADO DE LESAO CORPORAL GRAVE'
]

In [12]:
crimes_comercio = [
    'FURTO SIMPLES',
    'FURTO QUALIFICADO',
    'ROUBO',
    'DANO',
    'ROUBO AGRAVADO',
    'VIOLACAO DE DOMICILIO',
    'FURTO DE COISA COMUM',
    'ROUBO COM RESULTADO DE LESAO CORPORAL GRAVE',
]

df_crimes_comercio = dt[base[0]][dt[base[0]]['Natureza'].isin(crimes_comercio)]


In [42]:
def calcular_indice_bairros(df, crimes_perigosos):
    """
    Calcula o índice de criminalidade por 100 mil habitantes para cada bairro
    a partir de um DataFrame 'df' que contém:
    - Bairro
    - Populacao
    (os registros representam ocorrências de crimes individuais)

    Filtra apenas os crimes presentes em 'crimes_perigosos'.

    Retorna um DataFrame com:
    - Bairro
    - Crime (quantidade de ocorrências)
    - Populacao
    - Indice (por 100 mil habitantes)
    - Percentual do total de crimes
    """

    # Filtra apenas crimes perigosos
    df = df[df['Natureza'].isin(crimes_perigosos)]

    # Verifica se as colunas necessárias existem
    colunas_necessarias = {'Bairro', 'Populacao'}
    if not colunas_necessarias.issubset(df.columns):
        raise ValueError(f"O DataFrame deve conter as colunas: {colunas_necessarias}")

    # Contagem de crimes por bairro
    ranking = df['Bairro'].value_counts().reset_index(name='Crime')
    ranking.columns = ['Bairro', 'Crime']

    # População de cada bairro (removendo duplicadas)
    pop = df[['Bairro', 'Populacao']].drop_duplicates()

    # Merge da contagem com população
    df_final = ranking.merge(pop, on='Bairro', how='left')

    # Índice por 100 mil habitantes
    df_final['Indice(100 mil/hab)'] = (df_final['Crime'] / df_final['Populacao']) * 100000

    # Ordena do maior para o menor índice
    df_final = df_final.sort_values(by='Indice(100 mil/hab)', ascending=False).reset_index(drop=True)

    return df_final



In [54]:
# Ranking 20 bairros mais ocorrencias de crimes perigosos

df_bairros_perigosos = df_crimes_perigosos['Bairro'].value_counts().reset_index(name='crimes')
df_bairros_perigosos.columns = ['Bairro', 'Crimes']

# Exibe os 10 primeiros bairros
print(df_bairros_perigosos.head(20))

                           Bairro  Crimes
0                          CENTRO   10366
1   CIDADE INDUSTRIAL DE CURITIBA    3793
2                      AGUA VERDE    3013
3                   SITIO CERCADO    2504
4                       BOQUEIRAO    2460
5                        REBOUCAS    2299
6                          PORTAO    2000
7                          CAJURU    1785
8                       TATUQUARA    1783
9                     PINHEIRINHO    1733
10                          BATEL    1685
11                        UBERABA    1547
12                          HAUER    1271
13                     NOVO MUNDO    1193
14                     CAPAO RASO    1150
15                JARDIM BOTANICO    1119
16                  SAO FRANCISCO    1103
17                          XAXIM    1076
18                     BIGORRILHO    1043
19                     CRISTO REI     924


In [55]:
# Ranking 20 bairros com mais indíce de criminalidade

resultado_bairros = calcular_indice_bairros(dt[base[0]], crimes_perigosos)
print(resultado_bairros.head(20))

                 Bairro  Crime  Populacao  Indice(100 mil/hab)
0                CENTRO  10366    39970.0         25934.450838
1         SAO FRANCISCO   1103     4974.0         22175.311620
2       JARDIM BOTANICO   1119     5475.0         20438.356164
3                 BATEL   1685    10027.0         16804.627506
4              REBOUCAS   2299    13998.0         16423.774825
5           PRADO VELHO    787     4865.0         16176.772867
6         CENTRO CIVICO    728     4544.0         16021.126761
7            ALTO DA XV    907     8252.0         10991.274842
8                 HAUER   1271    11935.0         10649.350649
9        ALTO DA GLORIA    553     6540.0          8455.657492
10           HUGO LANGE    352     4443.0          7922.574837
11              PAROLIN    768     9941.0          7725.580927
12               TARUMA    520     7108.0          7315.700619
13               MERCES    798    12523.0          6372.275014
14           BOM RETIRO    314     5112.0          6142